# PC Session 4

**Author:**
[Helge Liebert](https://hliebert.github.io/)

## Model inputs

In [ ]:
# generate random data in which y is a noisy function of x
x1 <- runif(1000, -5, 5)
x2 <- runif(1000, -15, 15)
x3 <- runif(1000, -1, 7)
intercept <- rep(1, 1000)

In [ ]:
## Outcome and matrix of regressors plus intercept, univariate case
y <- x1 + rnorm(1000) + 3
X <- as.matrix(cbind(intercept, x1))

In [ ]:
## Outcome and matrix of regressors plus intercept, multivariate case
## y <- 3*x1 + 0.5*x2 + 15*x3  + rnorm(1000) + 3
## X <- as.matrix(cbind(intercept, x1, x2, x3))

In [ ]:
length(y)
dim(X)
cbind(X,y)

## Different ways of computing linear regression

### Linear regression by solving normal equations

In [ ]:
## Using matrices to solve the system of equations, (X'X)^-1 X'y
b.ols <- solve(t(X) %*% X) %*% t(X) %*% y
b.ols 

In [ ]:
# more idiomatic
b.ols <- solve(crossprod(X), crossprod(X,y))
b.ols

### Linear regression by QR-decomposition

In [ ]:
## Fit a linear model, QR-decomposition/Gram-Schmidt orthogonalization
b.qrd <- lm(y ~ X - 1)
b.qrd

In [ ]:
## Doing qrd manually and solving
qrdecomp <- qr(X)
b.qrd <- as.matrix(backsolve(qrdecomp$qr, qr.qty(qrdecomp, y)))
b.qrd

### Linear regression by SVD

In [ ]:
svdecomp <- svd(X)
str(svdecomp)

In [ ]:
## computing A^{+}y, (X'X)^{-1}X' is Moore-Penrose Pseudoinverse, optained via SVD, 
b.svd <- svdecomp$v %*% solve(diag(svdecomp$d)) %*% t(svdecomp$u) %*% y
b.svd

In [ ]:
## function for Moore-Penrose Pseudoinverse
b.svd <- MASS::ginv(X) %*% y
b.svd

In [ ]:
## Moore-penrose Pseudoinverse, computed via eigenvalue decomposition of x'x
egd <- eigen(t(X) %*% X)
xtx.inv <- egd$vectors %*% solve(diag(egd$values)) %*% t(egd$vectors)
x.pseudo.inv <- xtx.inv %*% t(X)
b.svd <- x.pseudo.inv %*% y
b.svd

### Gradient descent

In [ ]:
## squared error cost function
cost <- function(X, y, theta) {
  sum((X %*% theta - y)^2) / (2 * length(y))
}

## euclidean norm
eucnorm <- function(x) sqrt(sum(x^2))

#### Batch gradient descent

Simple approach, fixed # of iterations

In [ ]:
## initialize coefficients
theta <- matrix(c(0, 0), nrow = 2) ## univariate
## theta <- matrix(c(0, 0), nrow = 4) ## multivariate

In [ ]:
## learning rate and iteration limit
alpha <- 0.01
niter <- 2000

In [ ]:
## keep history
cost_history <- double(niter)
theta_history <- list(niter)

In [ ]:
## compute gradient and update
set.seed(42)
for (i in 1:niter) {
  error <- (X %*% theta) - y
  delta <- t(X) %*% error / length(y)
  theta <- theta - alpha * delta
  cost_history[i] <- cost(X, y, theta)
  theta_history[[i]] <- theta
  ## if ((i %% 100) == 0) print(theta)
}
print(theta)
print(niter)

#### Batch gradient descent

With a stopping rule instead of a fixed number of iterations

In [ ]:
## initialize coefficients
theta <- matrix(c(0, 0), nrow = 2) ## univariate
## theta <- matrix(c(0, 0), nrow = 4) ## multivariate

In [ ]:
## keep history
cost_history <- c()
theta_history <- list()

In [ ]:
## stopping threshold
epsilon <- 10e-10
delta <- Inf
i = 0

In [ ]:
## compute gradient and update
## threshold and first iteration values
set.seed(42)
while (eucnorm(delta) > epsilon) {
  error <- (X %*% theta) - y
  delta <- t(X) %*% error / length(y)
  theta <- theta - alpha * delta
  cost_history <- c(cost_history, cost(X, y, theta))
  theta_history <- append(theta_history, list(theta))
  ## if ((i %% 100) == 0) print(theta)
  i <- i + 1
}
print(theta)
print(i)

#### Compare estimates

In [ ]:
## compare estimates
plot(x1, y, col = rgb(0.2, 0.4, 0.6, 0.4), 
     main = "Linear regression (by QR-decomp, normal eq, SVD or gradient descent)")
abline(b.qrd[1:2], col = "blue")
#abline(b.ols[1:2], col = "green")
#abline(b.svd[1:2], col = "red")
#abline(theta[1:2], col = "pink")

#### Converging fit

In [ ]:
## plot data and converging fit
plot(x1, y, col = rgb(0.2, 0.4, 0.6, 0.4),
     main = "Linear regression by gradient descent")
for (i in c(1, 3, 6, 10, 14, seq(20, niter, by = 10))) {
  abline(coef = theta_history[[i]], col = rgb(0.8, 0, 0, 0.3))
}
abline(coef = theta, col = "blue")

#### Cost function

In [ ]:
## cost convergence
plot(cost_history, type = "l", col = "blue", lwd = 2,
     main = "Cost function", ylab = "cost", xlab = "Iterations")

In [ ]:
conv <- as.data.frame(cbind(t(sapply(theta_history, function(x) x[, 1])), cost = cost_history))

library(plot3D)
scatter3D(
  x = conv$intercept,
  y = conv$x1,
  z = conv$cost,
  xlab = "intercept",
  ylab = "slope",
  zlab = "cost (mse)",
  col = ramp.col(
    col = sort(RColorBrewer::brewer.pal(9, "Blues"), decreasing = F),
    n = length(unique(conv$cost))
  ),
  colkey = F,
  phi = 10,
  theta = 45,
  main = "Gradient Descent (3D View)"
)

### Stochastic gradient descent, single obs 

In [ ]:
theta <- matrix(c(0, 0), nrow = 2) ## univariate
## theta <- matrix(c(0, 0), nrow = 4) ## multivariate
cost_history <- c()
theta_history <- list()
set.seed(42)
for (i in 1:niter) {
  j <- sample(NROW(X), 1)
  error <- (X[j, ] %*% theta) - y[j]
  delta <- X[j, ] %*% error / length(y[j])
  theta <- theta - alpha * delta
  cost_history[i] <- cost(X[j, ], y[j], theta) ## cost function could be simplified
  theta_history[[i]] <- theta
}
print(theta)

In [ ]:
## plot data and converging fit
plot(x1, y, col = rgb(0.2, 0.4, 0.6, 0.4),
     main = "Linear regression by stochastic gradient descent, single obs")
for (i in c(1, 3, 6, 10, 14, seq(20, niter, by = 10))) {
  abline(coef = theta_history[[i]], col = rgb(0.8, 0, 0, 0.3))
}
abline(coef = theta, col = "blue")

In [ ]:
## cost convergence
plot(cost_history, type = "l", col = "blue", lwd = 2,
     main = "Cost function", ylab = "cost", xlab = "Iterations")

In [ ]:
conv <- as.data.frame(cbind(t(sapply(theta_history, function(x) x[, 1])), cost = cost_history))

library(plot3D)
scatter3D(
  x = conv$intercept,
  y = conv$x1,
  z = conv$cost,
  xlab = "intercept",
  ylab = "slope",
  zlab = "cost (mse)",
  col = ramp.col(
    col = sort(RColorBrewer::brewer.pal(9, "Blues"), decreasing = F),
    n = length(unique(conv$cost))
  ),
  colkey = F,
  phi = 10,
  theta = 45,
  main = "Gradient Descent (3D View)"
)

### Stochastic gradient descent, single batch 

In [ ]:
theta <- matrix(c(0, 0), nrow = 2) ## univariate
## theta <- matrix(c(0, 0), nrow = 4) ## multivariate
cost_history <- c()
theta_history <- list()
set.seed(42)
for (i in 1:niter) {
  select <- sample(NROW(X), 32)
  error <- (X[select, ] %*% theta) - y[select]
  delta <- t(X[select, ]) %*% error / length(y[select])
  theta <- theta - alpha * delta
  cost_history[i] <- cost(X[select, ], y[select], theta)
  theta_history[[i]] <- theta
}
print(theta)

In [ ]:
## plot data and converging fit
plot(x1, y, col = rgb(0.2, 0.4, 0.6, 0.4),
     main = "Linear regression by stochastic gradient descent, mini batch")
for (i in c(1, 3, 6, 10, 14, seq(20, niter, by = 10))) {
  abline(coef = theta_history[[i]], col = rgb(0.8, 0, 0, 0.3))
}
abline(coef = theta, col = "blue")

In [ ]:
## cost convergence
plot(cost_history, type = "l", col = "blue", lwd = 2,
     main = "Cost function", ylab = "cost", xlab = "Iterations")

### Stochastic gradient descent, mini batch w/ multiple batches 

...